# Домашнее задание № 2 (часть 1)

## Морфология

##### Предварительная подготовка
Подключим исправления.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
from pprint import pprint

Я взяла произведение Гончарова "Фрегат"

In [3]:
file = "C:\\Users\\Андрей\\Downloads\\Гончаров. Фрегат.txt"
text = open(file, encoding="utf-8").read()

##### 1)Обработка книги с помощью mystem:
Нам нужно распарсить файл в формате txt с помощью mystem. Для этого применим метод analyze и замерим время работы для этой строчки. Затем разбор приведем к формату json и запишем в файл.

In [4]:
import json
import os
# подключаем mystem
from pymystem3 import Mystem
m = Mystem(mystem_bin="C:\\Users\\Андрей\\Downloads\\mystem.exe")

In [5]:
%time ana = m.analyze(text)
if os.path.exists(".\\mystem_anas.json"):
    mode = "w"
else:
    mode = "x"
with open(".\\mystem_anas.json", mode, encoding="utf-8") as f:
    json_anas = json.dump(ana, f, ensure_ascii=False)

Wall time: 24min 46s


##### 2)Обработка книги с помощью pymorphy


In [6]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [7]:
# nltk.download('punkt')
import nltk
from nltk.tokenize import word_tokenize

Токенизируем текст, разбираем c помощью pymorphy и записывания произведенных разборов список и словарь(для скорости).

In [8]:
tokens = word_tokenize(text)

In [9]:
lemmas = []
known_words = {}

In [10]:
%%timeit
for token in tokens:
    if token in known_words:
        lemmas.append(known_words[token])
    else:
        ana = morph.parse(token)
        lemmas.append(ana)
        known_words[token] = ana

1:2: E228 missing whitespace around modulo operator


59.9 ms ± 7.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Из списка с разборами из каждого первого разбора получим слово, лемму и часть речи и запишем эти 3 значения в мини-словари слов, которые в свою очередь, поместим в общий список. В итоге получится список из словарей.

In [11]:
# список, который мы потом переведем в json
list_of_lemmas = []
for lem in lemmas:
    first = lem[0]
    word_box = {}
    word_box["word"] = first.word
    word_box["lemma"] = first.normal_form
    word_box["POS"] = first.tag.POS
    list_of_lemmas.append(word_box)

Запишем список в файл в формате json

In [12]:
if os.path.exists(".\\pymorphy_anas.json"):
    mode = "w"
else:
    mode = "x"
with open(".\\pymorphy_anas.json", mode, encoding="utf-8") as f:
    json_anas = json.dump(list_of_lemmas, f, ensure_ascii=False)

##### 3) Вопросы по словам
###### 3.1 Доля слов для каждой части речи. 
Для этого воспользуемся списком, полученным благодяря pymorphy. Составим список из частей речи и преобразуем его в counter.

In [13]:
from collections import Counter

In [14]:
# список частей речи
pos_list = []
for item in list_of_lemmas:
    if item["POS"] is not None:
        pos_list.append(item["POS"])
words_num = len(pos_list)
pos_count = Counter(pos_list)

Распечатаем получившийся каунтер (при этом поделив кол-во слов данной части речи на общее кол-во слов)

In [15]:
print("{:<30}{}".format("Часть речи", "Доля в тексте"))
for key, value in pos_count.items():
    print("{:<30}{}".format(key, value/words_num))

Часть речи                    Доля в тексте
NOUN                          0.27681510832495515
PREP                          0.11840834269718296
NUMR                          0.007609359906952905
ADJF                          0.1174325309992706
INTJ                          0.0030161452480927317
ADVB                          0.06908549687543122
PRTF                          0.009028722376643601
PRCL                          0.04631655725747629
VERB                          0.12183846866560215
INFN                          0.022039545015474993
NPRO                          0.06712401679578922
GRND                          0.008220474303625288
CONJ                          0.10827567173300215
PRED                          0.005470459518599562
ADJS                          0.010595935103593747
PRTS                          0.004465077769235318
COMP                          0.004258087409072092


###### 3.2 Топ-20 (по частотности) глаголов и наречий
Опять же воспользуемся списком pymorphy. Сделаем список из глаголов и из наречий и преобразуем их в counter.

In [16]:
verb_list = []
adverb_list = []
for item in list_of_lemmas:
    if item["POS"] == "VERB":
        verb_list.append(item["lemma"].lower())
    elif item["POS"] == "ADVB":
        adverb_list.append(item["lemma"].lower())
verb_count = Counter(verb_list)
adverb_count = Counter(adverb_list)

Распечатываем:

In [17]:
# глаголы
print("Топ-20 глаголов:")
print("{:<30}{}".format("Глагол", "Доля в тексте"))
for verb, value in verb_count.most_common(20):
    print("{:<30}{}".format(verb, value/words_num))
# наречия
print("\n")
print("Топ-20 наречий:")
print("{:<30}{}".format("Наречие", "Доля в тексте"))
for adverb, value in adverb_count.most_common(20):
    print("{:<30}{}".format(adverb, value/words_num))

Топ-20 глаголов:
Глагол                        Доля в тексте
быть                          0.00923571273680683
сказать                       0.002582451160131685
мочь                          0.0017939164547479645
спросить                      0.00166577956512311
говорить                      0.0014982159402290693
видеть                        0.0014883592564117728
идти                          0.0013306523153350286
стать                         0.0011335186389890984
отвечать                      0.001103948587537209
смотреть                      0.0010349518008161333
знать                         0.0010349518008161333
хотеть                        0.0009758116979123544
сидеть                        0.0008476748082874997
пойти                         0.0006899678672107556
думать                        0.000640684448124273
послать                       0.000640684448124273
выйти                         0.00062097108048968
ходить                        0.0005914010290377905
жить         

##### 4) N-Gramms
Возьмем наши токены, которые мы уже делали ранее, приведем к нижнему регистру, оставим лишь слова. Затем составим список из биграмм и сделаем из них counter. То же самое проделаем с триграммами.

In [20]:
# биграммы
word_tokens = [w.lower() for w in tokens if w.isalpha()]
biwords = [b for b in nltk.bigrams(word_tokens)]
bigram_count = Counter(biwords)
# триграммы
triwords = [t for t in nltk.trigrams(word_tokens)]
trigram_count = Counter(triwords)

In [21]:
# биграммы
print("Топ-25 биграмм:")
print("{:<30}{}".format("Биграмма", "Кол-во в тексте"))
for bi, value in bigram_count.most_common(25):
    lex_bi = bi[0] + " " + bi[1]
    print("{:<30}{}".format(lex_bi, value))
print("\n")
# триграммы
print("Топ-25 триграмм:")
print("{:<30}{}".format("Триграмма", "Кол-во в тексте"))
for tri, value in trigram_count.most_common(25):
    lex_tri = tri[0] + " " + tri[1] + " " + tri[2]
    print("{:<30}{}".format(lex_tri, value))

Топ-25 биграмм:
Биграмма                      Кол-во в тексте
и не                          114
как будто                     102
и в                           89
то есть                       89
у нас                         70
что это                       70
спросил я                     68
не было                       68
и на                          64
у него                        61
и с                           59
я не                          59
может быть                    58
все это                       55
потому что                    55
между тем                     53
сказал он                     52
и все                         47
и мы                          46
что он                        44
так что                       44
отвечал он                    43
сказал я                      42
и я                           42
в самом                       41


Топ-25 триграмм:
Триграмма                     Кол-во в тексте
в самом деле                  37
до сих пор   